In [5]:
# Step 2: Simplify ONNX model (critical for quantized models)
from onnxsim import simplify
import onnx
from onnx2tf import convert
import tensorflow as tf

In [6]:
input_model = "/Users/yousufsarfraz_1/Documents/GitHub/mds-25/downsizing_experimental/det_500m.dynamic_int8_v1.onnx"
simplified_model = "/Users/yousufsarfraz_1/Documents/GitHub/mds-25/downsizing_experimental/quantized_model_simplified.onnx"

simplified_model_path = "/Users/yousufsarfraz_1/Documents/GitHub/mds-25/downsizing_experimental/quantized_model_simplified.onnx"

model = onnx.load(input_model)
simplified_model, check = simplify(model, 
                                 perform_optimization=True,
                                 skip_fuse_bn=False)
assert check, "Simplification failed"
onnx.save(simplified_model, simplified_model_path)



convert(
    input_onnx_file_path=simplified_model,
    output_folder_path="tflite_output",
    output_flatc_code=True,
    keep_shape_absolutely_input_names=None,
    enable_optimization=True,
    quantize_per_tensor=True,  # Preserve per-tensor quantization
    custom_input_op_name_np_data_paths={
        'serving_default_input:0': 'calibration_data.npy'  # Your calibration data
    }
)



interpreter = tf.lite.Interpreter(model_path="tflite_output/model_float32.tflite")
interpreter.allocate_tensors()

# Check input/output quantization parameters
input_details = interpreter.get_input_details()[0]
print(f"Input dtype: {input_details['dtype']}")  # Should show int8
print(f"Input scale: {input_details['quantization_parameters']['scales'][0]}")
print(f"Input zero point: {input_details['quantization_parameters']['zero_points'][0]}")

# Step 5: Final optimization (optional)
converter = tf.lite.TFLiteConverter.from_onnx_model(simplified_model)
converter._experimental_lower_tensor_list_ops = False  # Preserve quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

with open("final_quantized_model.tflite", "wb") as f:
    f.write(tflite_quant_model)


TypeError: simplify() got an unexpected keyword argument 'skip_fuse_ops'